In [1]:
import pandas as pd
import datetime

In [2]:
data_ = pd.read_csv('..\data\charts.csv')
data_.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0


In [3]:
data = data_.iloc[:]

In [4]:
# Artist
df_artist = pd.DataFrame(data['artist'].unique(), columns=['name'])
df_artist = df_artist.assign(name=df_artist['name'].str.split(', ')).explode('name')
df_artist = pd.DataFrame(df_artist['name'].unique(), columns=['name'])
df_artist.reset_index(drop = True, inplace=True)
df_artist.insert(0, 'artist_id', range(1, 1 + len(df_artist)))

# Song
df_song = pd.DataFrame(data['title'].unique(), columns=['title'])
df_song.insert(0, 'song_id', range(1, 1 + len(df_song)))

# Trend
df_trend = pd.DataFrame(data['trend'].unique(), columns=['trend'])
df_trend.insert(0, 'trend_id', range(1, 1 + len(df_trend)))

# Day
df_day = pd.DataFrame(data['date'].unique(), columns=['date'])
df_day.insert(0, 'day_id', range(1, 1 + len(df_day)))

# Region
df_region = pd.DataFrame(data['region'].unique(), columns=['name'])
df_region.insert(0, 'region_id', range(1, 1 + len(df_region)))

# Category
df_category = pd.DataFrame(data['chart'].unique(), columns=['name'])
df_category.insert(0, 'category_id', range(1, 1 + len(df_category)))


In [5]:
artistDict = df_artist.set_index('name').T.to_dict('records')[0]
songDict = df_song.set_index('title').T.to_dict('records')[0]
trendDict = df_trend.set_index('trend').T.to_dict('records')[0]
dayDict = df_day.set_index('date').T.to_dict('records')[0]
regionDict = df_region.set_index('name').T.to_dict('records')[0]
categoryDict = df_category.set_index('name').T.to_dict('records')[0]

In [6]:
batch = 10000
batches = int(len(data)/batch)
residual = len(data) - batches*batch

print(f"Data len: {len(data)} | Batch: {batch} | Batches: {batches} | Residual {residual}")

Data len: 26173514 | Batch: 10000 | Batches: 2617 | Residual 3514


In [7]:
df_chart_final = pd.DataFrame()
num = 0
# MAIN BATCHES
start = datetime.datetime.now().timestamp()
for j in range(batches):
    df_chart = pd.DataFrame(columns=[ 'title','rank', 'date', 'region', 'chart', 'trend', 'streams'])
    start = datetime.datetime.now().timestamp()
    for i in range(batch):
        df_row = data.iloc[num]
        title = songDict[df_row.iloc[0]]
        rank = df_row.iloc[1]
        date = dayDict[df_row.iloc[2]]
        region = regionDict[df_row.iloc[5]]
        chart = categoryDict[df_row.iloc[6]]
        trend = trendDict[df_row.iloc[7]]
        streams = df_row.iloc[8]
        row = [title, rank, date, region, chart, trend, streams]
        df_chart.loc[num] = row
    
        num +=1
        pass
    df_chart_final = pd.concat([df_chart_final, df_chart])
    
    print(f"Batch {j+1}/{batches} done [{datetime.datetime.now().timestamp()-start}]")

# RESIDUALS
start = datetime.datetime.now().timestamp()
df_chart = pd.DataFrame(columns=[ 'title','rank', 'date', 'region', 'chart', 'trend', 'streams'])
for i in range(residual):
    df_row = data.iloc[num]
    title = songDict[df_row.iloc[0]]
    rank = df_row.iloc[1]
    date = dayDict[df_row.iloc[2]]
    region = regionDict[df_row.iloc[5]]
    chart = categoryDict[df_row.iloc[6]]
    trend = trendDict[df_row.iloc[7]]
    streams = df_row.iloc[8]
    row = [title, rank, date, region, chart, trend, streams]
    df_chart.loc[num] = row

    num +=1
    pass
df_chart_final = pd.concat([df_chart_final, df_chart])

print(f"Batch residual done [{datetime.datetime.now().timestamp()-start}]")
df_chart_final.to_csv('../data/df_chart.csv', mode='w', index = False)
print(df_chart_final)

Batch 1/2617 done [15.32052493095398]
Batch 2/2617 done [16.126137018203735]
Batch 3/2617 done [14.715291976928711]
Batch 4/2617 done [13.689591884613037]
Batch 5/2617 done [18.30553412437439]
Batch 6/2617 done [16.327522039413452]
Batch 7/2617 done [16.825329780578613]
Batch 8/2617 done [15.026164054870605]
Batch 9/2617 done [17.228641033172607]
Batch 10/2617 done [13.229946851730347]
Batch 11/2617 done [14.924129962921143]
Batch 12/2617 done [12.713762998580933]
Batch 13/2617 done [14.923885107040405]
Batch 14/2617 done [12.616914987564087]
Batch 15/2617 done [13.999556064605713]
Batch 16/2617 done [13.555660963058472]
Batch 17/2617 done [13.67862319946289]
Batch 18/2617 done [13.626638889312744]
Batch 19/2617 done [13.674623012542725]
Batch 20/2617 done [13.639636039733887]
Batch 21/2617 done [13.557661056518555]
Batch 22/2617 done [13.490318059921265]
Batch 23/2617 done [12.40081000328064]
Batch 24/2617 done [13.153326034545898]
Batch 25/2617 done [13.926172018051147]
Batch 26/2617

In [30]:

df_songArtis_WIP = data[['title', 'artist']].drop_duplicates().reset_index().drop(columns = ['index'])
df_songArtis_WIP = df_songArtis_WIP.assign(artist=df_songArtis_WIP['artist'].str.split(', ')).explode('artist')
df_songArtis_WIP.reset_index(drop = True, inplace=True)
df_songArtist_final = pd.DataFrame()

batch = 10000
batches = int(len(df_songArtis_WIP)/batch)
residual = len(df_songArtis_WIP) - batches*batch

print(f"Data len: {len(df_songArtis_WIP)} | Batch: {batch} | Batches: {batches} | Residual {residual}")
num = 0
# MAIN BATCHES
for j in range(batches):
    df_songArtist = pd.DataFrame(columns=[ 'title','artist'])
    start = datetime.datetime.now().timestamp()
    for i in range(batch):
        df_row = df_songArtis_WIP.iloc[num]
        song_id = songDict[df_row.iloc[0]]
        artist_id = artistDict[df_row.iloc[1]]
        row = [song_id, artist_id]
        df_songArtist.loc[num] = row
        num +=1

    df_songArtist_final = pd.concat([df_songArtist_final, df_songArtist])
    print(f"Batch {j+1}/{batches} done [{datetime.datetime.now().timestamp()-start}]")

df_songArtist = pd.DataFrame(columns=[ 'title','artist'])
start = datetime.datetime.now().timestamp()
for i in range(residual):
    df_row = df_songArtis_WIP.iloc[num]
    #print(df_row)
    song_id = songDict[df_row.iloc[0]]
    artist_id = artistDict[df_row.iloc[1]]
    row = [song_id, artist_id]
    df_songArtist.loc[num] = row
    num +=1

df_songArtist_final = pd.concat([df_songArtist_final, df_songArtist])
print(f"Batch residual done [{datetime.datetime.now().timestamp()-start}]")
df_songArtist_final.to_csv('../data/df_songArtist.csv', mode='w', index = False)
print(df_songArtist_final)


Data len: 269465 | Batch: 10000 | Batches: 26 | Residual 9465


KeyboardInterrupt: 